<center>

# **Đồ án cuối kì**

# **Môn Lập trình cho khoa học dữ liệu**

## **Chủ đề: Các vụ án mạng ở Mỹ từ năm 2016 đến năm 2020**

</center>

<center>

# **Mục lục**

</center>

# **Các thư viện sử dụng**

In [21]:
# Libraries and Modules used
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


---

# **Mô tả tập dữ liệu**

- Tập dữ liệu này được thu thập từ [**Murder Accountability Project**](https://www.murderdata.org/p/data-docs.html), do Thomas Hargrove sáng lập và công bố. Nhìn chung, tập dữ liệu mô tả về các vụ án giết người được xảy ra ở khắp mọi bang trên nước Mỹ từ năm 1976 đến năm 2020.

- Tập dữ liệu này là một tập dữ liệu mở được công bố dưới bản quyền [***Creative Commons CC-BY license***](https://creativecommons.org/licenses/by/4.0/), cho phép sử dụng không hạn chế dưới mọi hình thức, miễn là người dùng giữ nguyên tập dữ liệu gốc.

- Tập dữ liệu bao gồm những vụ án mạng được tổng hợp từ báo cáo của FBI và dữ liệu của Freedom of Information Act về các vụ án mạng không được báo cáo cho Justice Department.

- Do tập dữ liệu có kích cỡ quá lớn (khoảng 290MB) nên nhóm chúng em chỉ lọc lấy những vụ án từ năm 2016 đến năm 2020.

---

# **Tiền xử lý dữ liệu**

## Đọc dữ liệu vào Dataframe

In [22]:
homi_df = pd.read_csv('homicide.csv')
homi_df

C:\Users\phuon\AppData\Local\Temp\ipykernel_5048\3899079463.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  homi_df = pd.read_csv('homicide.csv')


,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,StateName,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
0,201604266AL06201,"Tallapoosa, AL",AL06201,Alabama,Alexander City,Municipal police,MAP,Yes,2016,NaN,...,White,Unknown or not reported,"Firearm, type not stated",Relationship not determined,Circumstances undetermined,NaN,0,0,92517.0,Rural Alabama
1,201604265AL05003,"Marshall, AL",AL05003,Alabama,Arab,Municipal police,MAP,Yes,2016,NaN,...,White,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Other negligent handling of gun,NaN,0,0,92517.0,Rural Alabama
2,201604529AL04301,"Lee, AL",AL04301,Alabama,Auburn,Municipal police,MAP,Yes,2016,NaN,...,Black,Unknown or not reported,"Firearm, type not stated",Other family,Other - not specified,NaN,0,0,92517.0,"Auburn-Opelika, AL"
3,201604593AL04301,"Lee, AL",AL04301,Alabama,Auburn,Municipal police,MAP,Yes,2016,NaN,...,Black,Unknown or not reported,"Firearm, type not stated",Relationship not determined,Other - not specified,NaN,0,0,92517.0,"Auburn-Opelika, AL"
4,201604594AL04301,"Lee, AL",AL04301,Alabama,Auburn,Municipal police,MAP,Yes,2016,NaN,...,Black,Unknown or not reported,"Firearm, type not stated",Relationship not determined,Other - not specified,NaN,0,0,92517.0,"Auburn-Opelika, AL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92086,202003001WY01700,"Sheridan, WY",WY01700,Wyoming,Sheridan County,Sheriff,FBI,Yes,2020,WYOMIN,...,American Indian or Alaskan Native,Not of Hispanic origin,"Handgun - pistol, revolver, etc",Sister,Brawl due to influence of alcohol,NaN,2,0,83121.0,Rural Wyoming
92087,202003001WY01700,"Sheridan, WY",WY01700,Wyoming,Sheridan County,Sheriff,FBI,Yes,2020,WYOMIN,...,American Indian or Alaskan Native,Not of Hispanic origin,"Handgun - pistol, revolver, etc",Sister,Brawl due to influence of alcohol,NaN,2,0,83121.0,Rural Wyoming
92088,202005001WY01101,"Laramie, WY",WY01101,Wyoming,Cheyenne,Municipal police,FBI,Yes,2020,WYOMIN,...,White,Not of Hispanic origin,"Firearm, type not stated",Other - known to victim,Other arguments,NaN,0,0,83121.0,"Cheyenne, WY"
92089,202005001WY01100,"Laramie, WY",WY01100,Wyoming,Laramie County,Sheriff,FBI,Yes,2020,WYOMIN,...,White,Hispanic origin,"Handgun - pistol, revolver, etc",Boyfriend,Other arguments,NaN,0,0,83121.0,"Cheyenne, WY"


## Số dòng và số cột của tập dữ liệu

In [23]:
row = homi_df.shape[0]
col = homi_df.shape[1]
print('Số dòng:', row)
print('Số cột', col)

Số dòng: 92091
Số cột 31


## Ý nghĩa của mỗi dòng

Mỗi dòng là thông tin của 1 vụ án bao gồm thời gian, địa điểm và thông tin liên quan đến nạn nhân và hung thủ.

## Kiểm tra các dòng bị trùng lắp

In [24]:
duplicated = homi_df.duplicated()
duplicated_line = homi_df[duplicated == True]
print('Số dòng bị trùng lắp:', len(duplicated_line))
duplicated_line

Số dòng bị trùng lắp: 314


,ID,CNTYFIPS,Ori,State,Agency,Agentype,Source,Solved,Year,StateName,...,OffRace,OffEthnic,Weapon,Relationship,Circumstance,Subcircum,VicCount,OffCount,FileDate,MSA
1046,201607001AR07001,"Union, AR",AR07001,Arkansas,El Dorado,Municipal police,FBI,No,2016,NaN,...,Unknown,Unknown or not reported,"Firearm, type not stated",Relationship not determined,Circumstances undetermined,NaN,2,0,11818.0,Rural Arkansas
1482,201608003CA03300,"Riverside, CA",CA03300,California,Riverside County,Sheriff,FBI,Yes,2016,NaN,...,White,Not of Hispanic origin,"Handgun - pistol, revolver, etc",Other - known to victim,Circumstances undetermined,NaN,3,0,32017.0,"Riverside-San Bernardino, CA"
1579,201612001CA03604,"San Bernardino, CA",CA03604,California,Fontana,Municipal police,FBI,Yes,2016,NaN,...,White,Hispanic origin,"Firearm, type not stated",Acquaintance,Narcotic drug laws,NaN,1,1,21517.0,"Riverside-San Bernardino, CA"
2005,201601002CA04204,"Santa Barbara, CA",CA04204,California,Santa Maria,Municipal police,FBI,No,2016,NaN,...,Unknown,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Circumstances undetermined,NaN,1,0,71316.0,"Santa Barbara-Santa Maria, CA"
2007,201601003CA04204,"Santa Barbara, CA",CA04204,California,Santa Maria,Municipal police,FBI,No,2016,NaN,...,Unknown,Unknown or not reported,"Handgun - pistol, revolver, etc",Relationship not determined,Circumstances undetermined,NaN,1,0,71316.0,"Santa Barbara-Santa Maria, CA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90792,202001002VA11900,"Petersburg, VA",VA11900,Virginia,Petersburg,Municipal police,FBI,No,2020,VIRGIN,...,Unknown,Unknown or not reported,Rifle,Relationship not determined,Other,NaN,1,0,83121.0,"Richmond, VA"
90909,202006004VA12200,"Richmond, VA",VA12200,Virginia,Richmond,Municipal police,FBI,Yes,2020,VIRGIN,...,Black,Not of Hispanic origin,"Handgun - pistol, revolver, etc",Relationship not determined,Circumstances undetermined,NaN,1,0,83121.0,"Richmond, VA"
91495,202010002WA03700,"Whatcom, WA",WA03700,Washington,Whatcom County,Sheriff,FBI,Yes,2020,WASHIN,...,White,Not of Hispanic origin,"Handgun - pistol, revolver, etc",Daughter,Other,NaN,1,0,83121.0,"Bellingham, WA"
91559,202012001WV02000,"Kanawha, WV",WV02000,West Virginia,Kanawha County,Sheriff,FBI,Yes,2020,WEST V,...,White,Unknown or not reported,"Firearm, type not stated",Acquaintance,Other,NaN,3,1,83121.0,"Charleston, WV"


## Ý nghĩa của mỗi cột

In [8]:
des_df = pd.read_csv('description.csv')
des_df

,Đặc trưng,Mô tả
0,ID,ID của vụ án


## Kiểm tra kiểu dữ liệu của các cột và điều chỉnh cho phù hợp

In [25]:
print(homi_df.shape[0])
for i in homi_df.columns:
    print(i)
    print(homi_df[i].nunique())

92091
ID
86890
CNTYFIPS
2384
Ori
7048
State
52
Agency
5589
Agentype
8
Source
2
Solved
2
Year
5
StateName
51
Month
12
Incident
1000
ActionType
2
Homicide
2
Situation
6
VicAge
101
VicSex
3
VicRace
6
VicEthnic
3
OffAge
100
OffSex
3
OffRace
6
OffEthnic
3
Weapon
17
Relationship
29
Circumstance
32
Subcircum
7
VicCount
13
OffCount
16
FileDate
389
MSA
408


## Tìm sự phân phối của các cột dữ liệu kiểu số

In [26]:
col_num = ['Year','Incident','VicAge','OffAge','VicCount','OffCount']

num_df = homi_df[col_num]

mising_percentage = (num_df.isna().sum() / row) * 100       # tỉ lệ giá trị bị thiếu
min_values = num_df.min()                                   # các giá trị nhỏ nhất
max_values = num_df.max()                                   # các giá trị lớn nhất

num_cols_distribution = pd.DataFrame(data = np.array((mising_percentage, min_values, max_values)),
                                                      columns = num_df.columns,
                                                      index = ['mising percentage', 'min', 'max'])
num_cols_distribution

,Year,Incident,VicAge,OffAge,VicCount,OffCount
mising percentage,0.0,0.0,0.0,0.0,0.0,0.0
min,2016.0,0.0,0.0,0.0,0.0,0.0
max,2020.0,999.0,999.0,999.0,21.0,40.0


## Tìm sự phân phối của các cột dữ liệu kiểu phân loại

## Xử lý các dòng bị thiếu dữ liệu

---

# **Phân tích dữ liệu**

## **Câu hỏi số 1**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

## **Câu hỏi số 2**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

## **Câu hỏi số 3**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

## **Câu hỏi số 4**

- (Nhập câu hỏi)
- (Lợi ích của việc tìm câu trả lời cho câu hỏi này)

### Tiền xử lý

### Phân tích

### Trực quan hóa

### Trả lời câu hỏi

- (Nhập câu trả lời và giải thích)

---

# **Đôi điều về đồ án**

- Mỗi thành viên gặp khó khăn gì?
- Mỗi thành viên học được những gì?
- Nếu có thêm thời gian, sẽ làm những gì?

---

# **Các nguồn tham khảo**

https://www.kaggle.com/code/nevil7/homicide-data-identifying-the-serial-killers